In [23]:
import os
import random
from loguru import logger
import json
save_dir = '/Users/keyi/Desktop/ds_11_25_multilabel_8tries'
train_dir_name = "per_sample_1_train"
pos_patches_per_file = 40
total_patches_per_file = 50
pos_samples_per_batch = 2
total_healthy_patches_per_patient = 2
neg_samples_per_batch = 1
split_number = 1

JSON_file = f'/Users/keyi/Desktop/DL_template/_assets/split_json/AutoPET/dataset_split_{split_number}.json'
def collect_healthy_indices():
    healthy_indices = []
    with open(JSON_file, 'r') as f:
        data = json.load(f)
        training = data['TRAINING']
        for idx, entry in enumerate(training):
            if entry['DIAGNOSIS'] == "NEGATIVE":  # healthy
                healthy_indices.append(idx)
    logger.info(f"found {len(healthy_indices)} healthy patients")
    return healthy_indices

healthy_indices = collect_healthy_indices()
training_numbers = 6

def _generate_train_combinations():
    used_patches = set() 
    combinations_batch = []
    # file_list = os.listdir(os.path.join(save_dir,  train_dir_name))
    # file_list = sorted([f for f in file_list if f.endswith('train.pt')])
    # print(f"file_list: {file_list}")
    
    healthy_neg_samples = []
    for healthy_idx in  healthy_indices:
        patches_indices = [(healthy_idx, i) for i in range( total_patches_per_file)]
        selected_indices = random.sample(patches_indices, 10)  # 10
        healthy_neg_samples.extend(selected_indices)
        logger.info(f"healthy_neg_samples: {healthy_neg_samples}")
        logger.info(f"Fixed {len(selected_indices)} negative samples for healthy patient {healthy_idx}")
    
    # first step: generate positive and negative indices for each patient
    # for file_idx, _ in enumerate(file_list):
    for file_idx in range(training_numbers):
        if file_idx not in  healthy_indices: 
            pos_indices = [(file_idx, i) for i in range( pos_patches_per_file)] 
            neg_indices = [(file_idx, i) for i in range( pos_patches_per_file,  total_patches_per_file)] 
            
            
            available_pos = [idx for idx in pos_indices if idx not in used_patches]
            # 10 negative samples
            for _ in range(total_patches_per_file - pos_patches_per_file):
                available_pos = [idx for idx in pos_indices if idx not in used_patches]
                available_neg = [idx for idx in neg_indices if idx not in used_patches]
                
                if len(available_pos) >= pos_samples_per_batch and available_neg:
                    pos_samples = random.sample(available_pos, pos_samples_per_batch)
                    neg_sample = random.choice(available_neg)
                    
                    used_patches.update(pos_samples)
                    used_patches.add(neg_sample)
                    
                    combinations_batch.append({
                        'positive': tuple(pos_samples),
                        'negative': (neg_sample)
                    })
    
    while len(healthy_neg_samples) > 0:
        sick_patients = [idx for idx in range(training_numbers) if idx not in healthy_indices]
        
        # Check if we have enough positive samples available for any sick patient
        has_enough_samples = False
        available_sick_patients = []
        for patient in sick_patients:
            pos_indices = [(patient, i) for i in range(pos_patches_per_file)]
            available_pos = [idx for idx in pos_indices if idx not in used_patches]
            if len(available_pos) >= pos_samples_per_batch:
                available_sick_patients.append(patient)
                has_enough_samples = True
        
        # If no sick patient has enough samples, break the loop
        if not has_enough_samples:
            logger.warning(f"Stopping early: No sick patients have enough positive samples. "
                        f"Remaining healthy samples: {len(healthy_neg_samples)}")
            break
        
        # Choose from patients that have enough samples
        random_chosen_sick_patient = random.choice(available_sick_patients)
        random_chosen_healthy_neg_sample = healthy_neg_samples.pop()
        
        pos_indices = [(random_chosen_sick_patient, i) for i in range(pos_patches_per_file)]
        available_pos = [idx for idx in pos_indices if idx not in used_patches]
        
        pos_samples = random.sample(available_pos, pos_samples_per_batch)
        used_patches.update(pos_samples)
        combinations_batch.append({
            'positive': tuple(pos_samples),
            'negative': (random_chosen_healthy_neg_sample)
        })
                    
    print(f"Generated {len(combinations_batch)} combinations")
    with open(f'combinations_batch_split_{split_number}.json', 'w') as f:
        json.dump(combinations_batch, f, indent=4)
    return combinations_batch
c = _generate_train_combinations()
for i, combo in enumerate(c):

     print(f"{i}: {combo}")
     
     
# 356 healthy samples
# 708 - 356 = 352 sick samples
  

2024-11-28 23:28:05.154 | INFO     | __main__:collect_healthy_indices:23 - found 4 healthy patients
2024-11-28 23:28:05.155 | INFO     | __main__:_generate_train_combinations:41 - healthy_neg_samples: [(0, 39), (0, 23), (0, 28), (0, 25), (0, 38), (0, 47), (0, 19), (0, 24), (0, 18), (0, 14)]
2024-11-28 23:28:05.155 | INFO     | __main__:_generate_train_combinations:42 - Fixed 10 negative samples for healthy patient 0
2024-11-28 23:28:05.156 | INFO     | __main__:_generate_train_combinations:41 - healthy_neg_samples: [(0, 39), (0, 23), (0, 28), (0, 25), (0, 38), (0, 47), (0, 19), (0, 24), (0, 18), (0, 14), (3, 1), (3, 20), (3, 47), (3, 13), (3, 21), (3, 10), (3, 32), (3, 31), (3, 35), (3, 3)]
2024-11-28 23:28:05.156 | INFO     | __main__:_generate_train_combinations:42 - Fixed 10 negative samples for healthy patient 3
2024-11-28 23:28:05.157 | INFO     | __main__:_generate_train_combinations:41 - healthy_neg_samples: [(0, 39), (0, 23), (0, 28), (0, 25), (0, 38), (0, 47), (0, 19), (0, 24)

Generated 40 combinations
0: {'positive': ((1, 35), (1, 32)), 'negative': (1, 40)}
1: {'positive': ((1, 18), (1, 11)), 'negative': (1, 41)}
2: {'positive': ((1, 22), (1, 36)), 'negative': (1, 47)}
3: {'positive': ((1, 37), (1, 31)), 'negative': (1, 42)}
4: {'positive': ((1, 16), (1, 26)), 'negative': (1, 46)}
5: {'positive': ((1, 27), (1, 9)), 'negative': (1, 43)}
6: {'positive': ((1, 33), (1, 10)), 'negative': (1, 44)}
7: {'positive': ((1, 13), (1, 1)), 'negative': (1, 45)}
8: {'positive': ((1, 14), (1, 4)), 'negative': (1, 49)}
9: {'positive': ((1, 0), (1, 24)), 'negative': (1, 48)}
10: {'positive': ((2, 17), (2, 6)), 'negative': (2, 44)}
11: {'positive': ((2, 26), (2, 39)), 'negative': (2, 48)}
12: {'positive': ((2, 28), (2, 11)), 'negative': (2, 42)}
13: {'positive': ((2, 2), (2, 23)), 'negative': (2, 46)}
14: {'positive': ((2, 31), (2, 3)), 'negative': (2, 45)}
15: {'positive': ((2, 21), (2, 29)), 'negative': (2, 43)}
16: {'positive': ((2, 34), (2, 32)), 'negative': (2, 41)}
17: {

In [1]:
import json
with open("/Users/keyi/Desktop/DL_template/_assets/split_training/AutoPET_Cluster/combinations_batch_split_1.json", "r") as f:
    data = json.load(f)
    print(len(data))
    pos_1 = data[0]["positive"][0]
    pos_2 = data[0]["positive"][1]
    neg = data[0]["negative"]
    print(pos_1, pos_2, neg)
    print(pos_1[0])
    print(pos_1[1])



6940
[0, 36] [0, 31] [0, 46]
0
36
